## Code playground for generating query expansions based on embedding model

##Format Input Queries and Paragraphs

In [3]:
#     broadQuery = independentRelevance(testQuery)
#     fineQuery = independentRelevanceFine(testQuery)
    
#     # If there are no results, skip the remaining checks
#     if(fineQuery == [] and broadQuery == ['',[]]):
#         print("This query \"" + testQuery + "\" does not have any relevant expansions")
#         print("--------------------------------------------------------")
# #         continue
#     # If there is a broad query, print it
#     # This can still occur even if there are no fine results?
#     if(broadQuery != ['',[]]):
#         print("\n\033[1m Broad: \033[0m\'" + testQuery + "\" = \n" + '\n'.join(str(broadQuery[1]).split(",")))
#     else:
#         print("There are no broad results for this query")

#     # If there is a fine query found from the broadQuery results, print it
#     if(fineQuery != {}):
#         print("\n\033[1m Fine : \033[0m\'"+ testQuery + "\" = \n" + ' '.join(str(fineQuery).split(",")))
#     else:
#         print("There are no fine results using the broadly relevant query words")
#     print("--------------------------------------------------------")


In [ ]:
maxTokenLength = 0
def tsne_plot(tokens_input,maxTokenLength):
    "Creates and TSNE model and plots it"
    labels = []
    tokens = []
    
    import tensorflow as tf
    import sys
    np.set_printoptions(threshold=sys.maxsize)

#     tokens_input = [["agriprocessors", "bankruptcy", "", "", ""],
#                 ["agriprocessors","controversies","","",""],
#                 ["animal","agriprocessors","abuse","controversies",""],
#                 ["agriprocessors","controversies","federal","immigration","raid"],
#                 ["agriprocessors","","","",""]]
    
    tokens_length = [len(x) for x in tokens_input]
    
    maxTokenLength = max(tokens_length)
    
    
    for x in tokens_input:
        if(len(x) < maxTokenLength):
            for y in range(maxTokenLength - len(x)):
                x.append("")
    print(tokens_input)
            
    embeddings = elmo(inputs={"tokens": tokens_input,"sequence_len": tokens_length},
                      signature="tokens",
                      as_dict=True)["elmo"]

    init = tf.initialize_all_variables()
    sess = tf.Session()
    sess.run(init)
    tokens_input = [list(filter(None,x)) for x in tokens_input if x]
    for x in range(len(tokens_input)):
        sent = tokens_input[x]
        for i in range(len(sent)):
            tokens.append(sess.run(embeddings[x][i]))
            labels.append(sent[i])  

    tsne_model = TSNE(perplexity=6, n_components=2, init='random', n_iter=500)
    new_values = tsne_model.fit_transform(tokens,verbose=1)

    x = []
    y = []
    for value in new_values:
        x.append(value[0])
        y.append(value[1])
        
    plt.figure(figsize=(18, 12)) 
    for i in range(len(x)):
        plt.scatter(x[i],y[i])
        plt.annotate(labels[i],
                     xy=(x[i], y[i]),
                     xytext=(5, 2),
                     textcoords='offset points',
                     ha='right',
                     va='bottom')
    plt.show()


In [ ]:
# print(splitLine,maxTokenLength)
# tsne_plot(splitLine,maxTokenLength)
# print("Plot created")

In [9]:
print("Functions Imported")

Functions Imported


In [ ]:
# Function that takes the paragraph associated with a query
# Returns the unique words without stopwords
def formatQueryParagraph(queryText,stem):
    queryParagraph = [x.strip() for x in queryText.split(" ") if x not in stopWords]
    if(stem):
        return ' '.join(set(stemmer.stem(word) for word in queryParagraph))
    return ' '.join(set(word for word in queryParagraph))

In [1]:
# Function to create map of word to top n simiar words.
# This uses the embedding of each word to find the most similar words
# Returns a string mapping the top n words related to each word in the query.
# Removes any words that exist in the original query
def independentN(queryWords,stem,n):
    independentSimilar = ""
    independentSimilarList = []
    queryWords = queryWords.split(" ")
#     print(queryWords)
    vocab = glove_model.wv.vocab
#     print([word for word in queryWords])
#     print([word in vocab for word in queryWords])
    similarMap = [(word,glove_model.most_similar(positive=[word],topn=n)) for word in queryWords if word in vocab]
#     print(similarMap)
#     print(similarMap)
    for key,value in similarMap:
        if(key in vocab):
            if(stem):
                independentSimilar = key +':' + ' '.join(set(stemmer.stem(x[0]) for x in value if x[0] not in queryWords))
            else:
                independentSimilar = key +':' + ' '.join(set(x[0] for x in value if x[0] not in queryWords))
            independentSimilarList.append(independentSimilar)
        else:
            continue
    return independentSimilarList

In [1]:
# testParagraphFile = "../Galago/data/paragraphCorpus.txt"
# sdmQueryFile = "../Galago/Queries/sdm/rih-query.json"

# QrelMap is a map of qrel ID to 
def loadData (qrelMap, qrelTestMap, queryText,splitLine):
    qrelText = "../Java_API/Car-Grpc/src/main/resources/data/qrelMap-train.txt"
    qrelTestText = "../Java_API/Car-Grpc/src/main/resources/data/qrelMap-test.txt"
#     queryFile = "../Java_API/Car-Grpc/src/main/resources/Queries/Glove/rih-0-combine-query.json"
    paragraphFile = "paragraphCorpus.txt"
    
    with open(paragraphFile,'r') as f:
        for line in f:
            splitLine.append(list(filter(None,' '.join(line.split("__PERIOD__")).split(" "))))

#     with open(queryFile,'r') as f:
#         p = re.compile("\((.*?) \)")
        
#         for line in f:
#             queryText += ' '.join([x.strip() for x in p.findall(line)])

    with open(qrelText,'r') as f:
        for line in f:
            test = line.split(":")
            
            key = formatQuery(test[1],False).strip()
    
            value = formatQueryParagraph(test[2],False).strip()
            if(key not in qrelMap):
                qrelMap[key] = [value]
            else:
                qrelMap[key].append(value)
                
    with open(qrelTestText,'r') as f:
        for line in f:
            test = line.split(":")
            
            key = formatQuery(test[1],False).strip()
    
            value = formatQueryParagraph(test[2],False).strip()
            if(key not in qrelTestMap):
                qrelTestMap[key] = [value]
            else:
                qrelTestMap[key].append(value)                

    print("loading complete")
#     return queryText

In [ ]:
# Function that takes a query Id in the form Aftertaste/Distinguishing%20aftertaste%20and%20flavor
# Returns aftertaste distinguishing aftertaste flavor
# Removes slashes, %20, and lower cases, as well as removing stopwords.
def formatQuery(queryId,stem):
    queryIdWords = [x.lower() for x in queryId.replace("/"," ").replace("%20"," ").split(" ") if x.lower() not in stopWords]
    if(stem == True):
        return ' '.join(set(stemmer.stem(word) for word in queryIdWords)).strip()
    else:
        return ' '.join(list(dict.fromkeys(word for word in queryIdWords))).strip()